In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
device

device(type='cuda', index=0)

In [3]:
data_directory = "../My_PyTorch_Data/gilbut/chap05/data"
train_dataset = torchvision.datasets.FashionMNIST(data_directory, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST(data_directory, download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))

c:\Users\rubin\anaconda3\envs\torch_book\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
train_loader = DataLoader(train_dataset, batch_size=100)
test_loader = DataLoader(test_dataset, batch_size=100)

In [5]:
labels_map = {0:'T-Shirt', 1:'Trouser', 2:'Pullover', 3:'Dress', 4:'Coat', 5:'Sandal', 6:'Shirt', 7:'Sneaker', 8:'Bag', 9:'AnkleBoot'}

In [6]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(num_features=64), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(p=0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [7]:
learning_rate = 0.001
model = FashionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [8]:
num_epochs = 5
count = 0
loss_lst, iteration_lst, accuracy_lst = [], [], []
predictions_lst, labels_lst = [], []

In [9]:
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
        
        if not (count % 50):
            total = 0
            correct = 0
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_lst.append(labels)
                test = Variable(images.view(100, 1, 28, 28))
                outputs = model(test)
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_lst.append(predictions)
                correct += (predictions == labels).sum()
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_lst.append(loss.data)
            iteration_lst.append(count)
            accuracy_lst.append(accuracy)
        
        if not (count % 500):
            print(f"Iteration : {count},  Loss : {loss.data},  Accuracy : {accuracy}")

c:\Users\rubin\anaconda3\envs\torch_book\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Iteration : 500,  Loss : 0.5042083859443665,  Accuracy : 88.07999420166016
Iteration : 1000,  Loss : 0.2919063866138458,  Accuracy : 88.47000122070312
Iteration : 1500,  Loss : 0.3153984844684601,  Accuracy : 87.45999908447266
Iteration : 2000,  Loss : 0.24187734723091125,  Accuracy : 89.38999938964844
Iteration : 2500,  Loss : 0.17871467769145966,  Accuracy : 89.18000030517578
Iteration : 3000,  Loss : 0.13069751858711243,  Accuracy : 89.52999877929688


In [14]:
len(loss_lst), loss_lst

(60,
 [tensor(0.5262, device='cuda:0'),
  tensor(0.5547, device='cuda:0'),
  tensor(0.5359, device='cuda:0'),
  tensor(0.3479, device='cuda:0'),
  tensor(0.3598, device='cuda:0'),
  tensor(0.3835, device='cuda:0'),
  tensor(0.3323, device='cuda:0'),
  tensor(0.4791, device='cuda:0'),
  tensor(0.6184, device='cuda:0'),
  tensor(0.5042, device='cuda:0'),
  tensor(0.3959, device='cuda:0'),
  tensor(0.2558, device='cuda:0'),
  tensor(0.3075, device='cuda:0'),
  tensor(0.3195, device='cuda:0'),
  tensor(0.3528, device='cuda:0'),
  tensor(0.3097, device='cuda:0'),
  tensor(0.2908, device='cuda:0'),
  tensor(0.3571, device='cuda:0'),
  tensor(0.2681, device='cuda:0'),
  tensor(0.2919, device='cuda:0'),
  tensor(0.5271, device='cuda:0'),
  tensor(0.3580, device='cuda:0'),
  tensor(0.3610, device='cuda:0'),
  tensor(0.2018, device='cuda:0'),
  tensor(0.2362, device='cuda:0'),
  tensor(0.1944, device='cuda:0'),
  tensor(0.3292, device='cuda:0'),
  tensor(0.2354, device='cuda:0'),
  tensor(0.2527

In [13]:
accuracy_lst

[tensor(79.4300, device='cuda:0'),
 tensor(84.4500, device='cuda:0'),
 tensor(84.2700, device='cuda:0'),
 tensor(85.1700, device='cuda:0'),
 tensor(85.6000, device='cuda:0'),
 tensor(86., device='cuda:0'),
 tensor(87.3700, device='cuda:0'),
 tensor(85.5700, device='cuda:0'),
 tensor(85.8200, device='cuda:0'),
 tensor(88.0800, device='cuda:0'),
 tensor(87.4100, device='cuda:0'),
 tensor(87.3900, device='cuda:0'),
 tensor(88.1900, device='cuda:0'),
 tensor(88.2400, device='cuda:0'),
 tensor(86.9000, device='cuda:0'),
 tensor(87.3800, device='cuda:0'),
 tensor(88.1600, device='cuda:0'),
 tensor(87.8200, device='cuda:0'),
 tensor(87.7600, device='cuda:0'),
 tensor(88.4700, device='cuda:0'),
 tensor(86.1500, device='cuda:0'),
 tensor(88.8400, device='cuda:0'),
 tensor(88.0800, device='cuda:0'),
 tensor(87.9600, device='cuda:0'),
 tensor(89.1900, device='cuda:0'),
 tensor(89.3700, device='cuda:0'),
 tensor(88.0400, device='cuda:0'),
 tensor(89.0400, device='cuda:0'),
 tensor(88.0200, device=